In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from scipy.stats import skew
import plotly.express as px
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import resample
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_blobs
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from math import log2, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn import metrics
from sklearn import preprocessing
from sklearn.svm import SVC,LinearSVC
from tqdm.notebook import tqdm_notebook as tqdm
def PieChart(data, x, y):
    count = Counter(data)
    count = pd.DataFrame({x:count.keys(), y:count.values()})
    fig = px.pie(count, x, y)
    fig.show()

In [2]:
# Load the data set as a Pandas DataFrame
test = pd.read_csv('../input/twitter-sentiment-analysis-hatred-speech/test.csv',encoding='latin-1')
train= pd.read_csv('../input/twitter-sentiment-analysis-hatred-speech/train.csv',encoding='latin-1')

In [3]:
#check the data frame 
train.head()

In [4]:
test.head()

In [5]:

#Check for Nulls
train.isnull().any()
test.isnull().any()

In [6]:
#Since the training set is labled to indicate negative and positive comments we can check them independintly
train[train['label'] == 0].head()

In [7]:
train[train['label'] == 1].head()

In [8]:
#Check the percentage of negative tweets 
PieChart(train['label'],'label','Number')

In [12]:
# Undersampling the postive tweets to match the negative ones
# Class count
count_class_0, count_class_1 = train.label.value_counts()

# Divide by class
df_class_0 = train[train['label'] == 0]
df_class_1 = train[train['label'] == 1]
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.label.value_counts())

df_test_under.label.value_counts().plot(kind='bar', title='Count (label)');

In [13]:
#Check the percentage of negative tweets 
PieChart(df_test_under['label'],'label','Number')

In [15]:
Tdf = df_test_under
X = Tdf['tweet']
target = Tdf['label']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 101, shuffle=True)

In [17]:
y_train.head()

In [18]:
print("Shape Of Training Set",X_train.shape)
print("Shape Of Testing Set",X_test.shape)
print("Shape Of Training Target",y_train.shape)
print("Shape Of Testing Target",y_test.shape)

In [19]:
#TF-IDF stands for “Term Frequency — Inverse Document Frequency”
#This is a technique to quantify words in the dataset which computes a score for each word to signify its importance
tfidf_vectoriser = TfidfVectorizer()
tfidf_vectoriser.fit(X_train)
X_train_tf = tfidf_vectoriser.transform(X_train)
X_test_tf = tfidf_vectoriser.transform(X_test)
X_train_tf.shape, X_test_tf.shape

In [20]:
svm = LinearSVC(max_iter=3000)
svm.fit(X_train_tf, y_train)

In [21]:
y_pred = svm.predict(X_test_tf)
print(classification_report(y_test, y_pred, target_names=['Negative','Positive']))
print("Accuracy :",accuracy_score(y_test, y_pred), end='\n\n')
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

In [22]:
C_range=list(np.arange(0.1,6,0.1))
acc_score=[]
for c in C_range:
    svc = LinearSVC(max_iter=1500,C=c)
    scores = cross_val_score(svc, X_train_tf, y_train, cv=10, scoring='accuracy')
    acc_score.append(scores.mean())
print(acc_score)   
# plot the value of C for SVM (x-axis) versus the cross-validated accuracy (y-axis)
plt.plot(C_range,acc_score)
plt.xticks(np.arange(0.0,6,0.3))
plt.xlabel('Value of C for SVC ')
plt.ylabel('Cross-Validated Accuracy')

In [23]:
param_grid = {
    'C':[0.6,0.71,0.72,0.73,0.74,0.75,0.76,0.77,0.78,0.79,0.8]
}
svm_model= LinearSVC(max_iter=1500)
model_svm = GridSearchCV(svm_model, param_grid,cv=10,scoring='accuracy')
model_svm.fit(X_train_tf, y_train)
print("Best Parameters :", model_svm.best_params_)
print("Best Score :",model_svm.best_score_)


In [25]:
best_model=model_svm.best_estimator_
coef = best_model.coef_.tolist()[0]
w = tfidf_vectoriser.get_feature_names()
coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
print('')
print('-Top 20 Negative-')
print(coeff_df.head(20).to_string(index=False))
print('')
print('-Top 20 Postive-')        
print(coeff_df.tail(20).to_string(index=False))

In [44]:
coeff_df.head(30).plot(x= 'Word', y='Coefficient', kind='bar', figsize=(15, 7), color = 'blue')
plt.title("Most Frequent Words in raceist/sexist tweets - Top 30")

In [ ]:
# Testing